In [1]:
# Cell 1: Core Components
from typing import List
import logging
from sentence_transformers import SentenceTransformer
import faiss
import subprocess
import time
import pickle
from datetime import datetime
import os

# ======== Retrieval Component ========
class RetrievalComponent:
    def __init__(self, config):
        self.config = config
        self.documents = []
        self.index = None
        self.retriever = None
        self.logger = logging.getLogger(__name__)
        self._init_retriever()

    def _init_retriever(self):
        self.retriever = SentenceTransformer(self.config['retriever_model'])

    def load_knowledge_base(self, file_path: str):
        try:
            with open(file_path, 'r') as f:
                self.documents = [line.strip() for line in f if line.strip()]
            
            embeddings = self.retriever.encode(self.documents)
            self.index = faiss.IndexFlatL2(embeddings.shape[1])
            self.index.add(embeddings.astype('float32'))
            self.logger.info(f"Loaded {len(self.documents)} documents")

        except Exception as e:
            self.logger.error(f"Knowledge base error: {str(e)}")
            raise

    def retrieve(self, query: str, k: int = 3) -> List[str]:
        try:
            query_embed = self.retriever.encode([query])
            _, indices = self.index.search(query_embed.astype('float32'), k)
            return [self.documents[i] for i in indices[0]]
        except Exception as e:
            self.logger.error(f"Retrieval error: {str(e)}")
            raise

# ======== Generation Component ========
class GenerationComponent:
    HISTORY_FILE = "ollama_rag_history.pkl"
    
    def __init__(self, config, retriever: RetrievalComponent):
        self.config = config
        self.retriever = retriever
        self.logger = logging.getLogger(__name__)

    def _run_ollama(self, prompt: str) -> str:
        response_data = {
            'timestamp': datetime.now().isoformat(),
            'model': self.config['model_name'],
            'prompt': prompt,
            'response': '',
            'execution_time': 0,
            'context': []
        }

        try:
            start_time = time.time()
            process = subprocess.run(
                ["ollama", "run", self.config['model_name']],
                input=prompt,
                capture_output=True,
                text=True,
                timeout=self.config.get('timeout', 60)
            )

            response = process.stdout.strip()
            elapsed_time = time.time() - start_time

            response_data.update({
                'response': response,
                'execution_time': round(elapsed_time, 2),
                'context': self.retriever.retrieve(prompt)
            })

            self._save_history(response_data)
            return response

        except subprocess.TimeoutExpired:
            response_data['response'] = "Error: Response timed out."
            self._save_history(response_data)
            return response_data['response']

    def _save_history(self, response_data: dict):
        try:
            history = {'interactions': []}
            if os.path.exists(self.HISTORY_FILE):
                with open(self.HISTORY_FILE, "rb") as f:
                    history = pickle.load(f)
            
            history['interactions'].append(response_data)
            
            with open(self.HISTORY_FILE, "wb") as f:
                pickle.dump(history, f)
                
        except Exception as e:
            self.logger.error(f"History save error: {str(e)}")

    def rag_generate(self, query: str) -> str:
        try:
            context = self.retriever.retrieve(query)
            prompt = f"Context: {' '.join(context)}\n\nQuestion: {query}\nAnswer:"
            return self._run_ollama(prompt)
        except Exception as e:
            self.logger.error(f"Generation error: {str(e)}")
            raise

    def show_history(self):
        try:
            with open(self.HISTORY_FILE, "rb") as f:
                history = pickle.load(f)
                print(f"\n{'='*40}\nRAG Interaction History")
                for idx, interaction in enumerate(history['interactions'], 1):
                    print(f"\nInteraction {idx} ({interaction['timestamp']}):")
                    print(f"Model: {interaction['model']}")
                    print(f"Context: {interaction['context'][:2]}...")
                    print(f"Prompt: {interaction['prompt'][:100]}...")
                    print(f"Response: {interaction['response'][:200]}...")
                    print(f"Execution time: {interaction['execution_time']}s")
        except FileNotFoundError:
            print("No history available")

In [2]:
# Cell 2: Usage Example
if __name__ == "__main__":
    # Configuration
    config = {
        'model_name': "hf.co/TheDrummer/Gemmasutra-Mini-2B-v1-GGUF:Q3_K_L",
        'retriever_model': "sentence-transformers/all-MiniLM-L6-v2",
        'timeout': 90,
        'max_context_items': 3
    }

    # Initialize components
    retriever = RetrievalComponent(config)
    retriever.load_knowledge_base("knowledge_base.txt")  # Replace with actual path
    
    generator = GenerationComponent(config, retriever)

    # Example interaction
    query = "Explain the benefits of running models locally."
    response = generator.rag_generate(query)
    print(f"\nQuery: {query}")
    print(f"Response: {response[:500]}...")  # Show first 500 characters

    # Display history
    print("\n=== Interaction History ===")
    generator.show_history()

Knowledge base error: [Errno 2] No such file or directory: 'knowledge_base.txt'


FileNotFoundError: [Errno 2] No such file or directory: 'knowledge_base.txt'